In [18]:
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

2022-12-13 13:45:08.318273: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-13 13:45:08.318323: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-13 13:45:19.859345: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-13 13:45:19.859430: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-13 13:45:19.859502: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (u): /proc/driver/nvidia/version does not exist


In [2]:
with open('../data/train.txt') as f:
    line = f.readline()
    line2 = f.readline()

In [3]:
def txt2array(path):
    txt, label = [], []
    with open(path) as f:
        while True:
            line = f.readline()
            if not line:
                break
            tmp = line.strip().split(';')
            if len(tmp)>1:
                txt.append(tmp[0])
                label.append(tmp[1])
            else:
                txt.append(tmp[0])
    if label:
        return txt, label
    else:
        return txt

### Converting Text file to Array

In [4]:
train_txt, train_label = txt2array('../data/train.txt')
val_txt, val_label = txt2array('../data/val.txt')
test_txt, test_label = txt2array('../data/test.txt')

### Array to DataFrame

In [5]:
train_dict = {"txt":train_txt, "label":train_label}
test_dict = {"txt":test_txt, "label":test_label}
val_dict = {"txt":test_txt, "label":test_label}

In [6]:
train_df = pd.DataFrame(train_dict)
test_df = pd.DataFrame(test_dict)
val_df = pd.DataFrame(val_dict)

In [7]:
tmp = train_df.label.unique()
unique_emo = {}
for i in range(len(tmp)):
    unique_emo[tmp[i]] = i    

In [8]:
train_df['label_num'] = train_df['label'].map(unique_emo)
test_df['label_num'] = test_df['label'].map(unique_emo)
val_df['label_num'] = val_df['label'].map(unique_emo)

In [9]:
train_df.head()

,txt,label,label_num
0,i didnt feel humiliated,sadness,0
1,i can go from feeling so hopeless to so damned...,sadness,0
2,im grabbing a minute to post i feel greedy wrong,anger,1
3,i am ever feeling nostalgic about the fireplac...,love,2
4,i am feeling grouchy,anger,1


In [10]:
x_train = train_df['txt']
y_train = train_df['label_num']

x_test = test_df['txt']
y_test = test_df['label_num']

x_val = val_df['txt']
y_val = val_df['label_num']

## Without Pre-processing Text data
### Random Forest


In [11]:
clf = Pipeline([('Vec_3grams', CountVectorizer(ngram_range=(3, 3))), #with tri-grams
               ('RF', RandomForestClassifier())])

clf.fit(x_train, y_train)

y_test_pred = clf.predict(x_test)

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.62      0.36      0.46       581
           1       0.48      0.20      0.29       275
           2       0.48      0.13      0.20       159
           3       0.47      0.14      0.21        66
           4       0.50      0.23      0.32       224
           5       0.44      0.87      0.58       695

    accuracy                           0.48      2000
   macro avg       0.50      0.32      0.34      2000
weighted avg       0.51      0.48      0.43      2000



In [12]:
clf = Pipeline([('Vec_3grams', CountVectorizer(ngram_range=(1, 2))), #with 1-grams and bi-grams
               ('RF', RandomForestClassifier())])

clf.fit(x_train, y_train)

y_test_pred = clf.predict(x_test)

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92       581
           1       0.92      0.83      0.87       275
           2       0.84      0.58      0.68       159
           3       0.75      0.55      0.63        66
           4       0.90      0.79      0.84       224
           5       0.80      0.96      0.88       695

    accuracy                           0.86      2000
   macro avg       0.86      0.77      0.80      2000
weighted avg       0.87      0.86      0.86      2000



In [13]:
clf = Pipeline([('TF-IDF', TfidfVectorizer()), #with TF-IDF
               ('RF', RandomForestClassifier())])

clf.fit(x_train, y_train)

y_test_pred = clf.predict(x_test)

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.93      0.90      0.91       581
           1       0.91      0.83      0.87       275
           2       0.81      0.64      0.71       159
           3       0.62      0.59      0.60        66
           4       0.87      0.82      0.84       224
           5       0.83      0.94      0.88       695

    accuracy                           0.86      2000
   macro avg       0.83      0.79      0.80      2000
weighted avg       0.87      0.86      0.86      2000



### Multinomial Naive Bayes

In [14]:
clf = Pipeline([('Vec_3grams', CountVectorizer(ngram_range=(3, 3))), #with tri-grams
               ('NB', MultinomialNB())])

clf.fit(x_train, y_train)

y_test_pred = clf.predict(x_test)

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.51      0.58      0.55       581
           1       0.59      0.17      0.27       275
           2       0.33      0.06      0.10       159
           3       0.50      0.03      0.06        66
           4       0.54      0.20      0.29       224
           5       0.48      0.80      0.60       695

    accuracy                           0.50      2000
   macro avg       0.49      0.31      0.31      2000
weighted avg       0.50      0.50      0.45      2000



In [16]:
clf = Pipeline([('TF-IDF', TfidfVectorizer()), #with TF-IDF
               ('NB', MultinomialNB())])

clf.fit(x_train, y_train)

y_test_pred = clf.predict(x_test)

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.67      0.90      0.77       581
           1       0.96      0.16      0.28       275
           2       1.00      0.03      0.05       159
           3       0.00      0.00      0.00        66
           4       1.00      0.18      0.31       224
           5       0.60      0.99      0.75       695

    accuracy                           0.65      2000
   macro avg       0.71      0.38      0.36      2000
weighted avg       0.73      0.65      0.56      2000



/home/s/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/s/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/s/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
nlp = spacy.load('en_core_web_md')

In [20]:
def Preprocess(txt):
    doc = nlp(txt)
    tmp = []
    for token in doc:
        if not token.is_stop or not token.is_punct:
            tmp.append(token.lemma_)
    return " ".join(tmp)

### With Preprocessing

In [23]:
x_train = x_train.apply(Preprocess)
x_test = x_test.apply(Preprocess)
x_val = x_val.apply(Preprocess)

### RF

In [24]:
clf = Pipeline([('TF-IDF', TfidfVectorizer()), #with TF-IDF
               ('RF', RandomForestClassifier())])

clf.fit(x_train, y_train)

y_test_pred = clf.predict(x_test)

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.91      0.87      0.89       581
           1       0.89      0.77      0.83       275
           2       0.81      0.57      0.67       159
           3       0.68      0.55      0.61        66
           4       0.85      0.84      0.85       224
           5       0.80      0.94      0.86       695

    accuracy                           0.84      2000
   macro avg       0.82      0.76      0.78      2000
weighted avg       0.85      0.84      0.84      2000



### NB

In [25]:
clf = Pipeline([('TF-IDF', TfidfVectorizer()), #with TF-IDF
               ('NB', MultinomialNB())])

clf.fit(x_train, y_train)

y_test_pred = clf.predict(x_test)

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.68      0.89      0.77       581
           1       0.93      0.20      0.33       275
           2       1.00      0.03      0.06       159
           3       0.00      0.00      0.00        66
           4       0.96      0.21      0.35       224
           5       0.61      0.99      0.76       695

    accuracy                           0.66      2000
   macro avg       0.70      0.39      0.38      2000
weighted avg       0.73      0.66      0.58      2000



/home/s/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/s/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/s/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
